### Advance FineTuning

In [1]:
#!pip install -U transformers datasets accelerate peft trl bitsandbytes wandb

In [2]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM


/Users/sidharthrao/Documents/Documents - Sidharth’s MacBook Pro/GitHub/Project-Dash/2. GenAI and AgenticAI/ii. Class Sessions/n. Advanced FineTuning/.venv_AdvFT/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from huggingface_hub import login
import os

# Login to Hugging Face
# Option 1: Use token from environment variable
hf_token = os.getenv("HF_TOKEN") or os.getenv("HUGGINGFACE_TOKEN")
if hf_token:
    login(token=hf_token)
else:
    # Option 2: Prompt for token (for interactive use)
    # login()  # This will prompt for token
    print("Please set HF_TOKEN or HUGGINGFACE_TOKEN environment variable, or use login() to enter token interactively")

In [4]:
model = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

In [5]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
print("Tokenizer loaded successfully!")

print("Loading model...")
model = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0", device_map="mps")
print("Model loaded successfully!")

# Test if model is working
print(f"Model device: {next(model.parameters()).device}")
print(f"Model dtype: {next(model.parameters()).dtype}")

Loading tokenizer...
Tokenizer loaded successfully!
Loading model...
Model loaded successfully!
Model device: mps:0
Model dtype: torch.float32


In [6]:
from transformers import pipeline

pipe = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0")
messages = [
    {"role": "user", "content": "How can I reset my password?	"},
]
pipe(messages)

Device set to use mps:0


[{'generated_text': [{'role': 'user',
    'content': 'How can I reset my password?\t'},
   {'role': 'assistant',
    'content': 'To reset your password, follow these steps:\n\n1. Login to your PayPal account.\n2. Go to your account settings.\n3. Scroll down to the "Password" section, and click on "Change Password."\n4. Enter your current password in the "Current Password" field.\n5. Enter your new password in the "New Password" field.\n6. Enter your confirmation password in the "Confirm Password" field.\n7. Click on the "Change Password" button.\n8. PayPal will generate a new password for you.\n\nNote: If you do not remember your current password, you can also use the "Forgot Password" feature to generate a new one.\n\nOnce you have reset your password, you will be prompted to verify your account by logging in with your new password.'}]}]

#### 

In [7]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)

'NoneType' object has no attribute 'cadam32bit_grad_fp32'


/Users/sidharthrao/Documents/Documents - Sidharth’s MacBook Pro/GitHub/Project-Dash/2. GenAI and AgenticAI/ii. Class Sessions/n. Advanced FineTuning/.venv_AdvFT/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [8]:
from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir="./llama-chatbot-finetuned",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,
    learning_rate=2e-4,
    #fp16=True,
    # evaluation_strategy="steps",
    save_strategy="steps",
    save_steps=100,
    logging_steps=10,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


#### Supervised Fine-Tuning Trainer Setup use the SFTTrainer or Trainer class for supervised fine-tuning

In [9]:
df = pd.read_csv("/Users/sidharthrao/Documents/Documents - Sidharth’s MacBook Pro/GitHub/Project-Dash/2. GenAI and AgenticAI/ii. Class Sessions/n. Advanced FineTuning/instruction-response.csv")
df.head()

,instruction,response
0,How can I reset my password?,"To reset your password, go to the login page a..."
1,What are your store hours?,Our store is open from 9 AM to 9 PM Monday thr...
2,Can you help me track my order?,"Yes, provide me with your order number and I'l..."
3,What is the refund policy?,Our refund policy allows returns within 30 day...
4,How do I contact customer service?,mailto:support@store.com


In [10]:
dataset = Dataset.from_pandas(df)

In [11]:
def concat_instruction_response(example):
    return {"text": f"Instruction: {example['instruction']}\nResponse: {example['response']}"}

dataset = dataset.map(concat_instruction_response)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Map: 100%|██████████| 10/10 [00:00<00:00, 2722.69 examples/s]


In [12]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    args=training_args,
)

Truncating train dataset: 100%|██████████| 10/10 [00:00<00:00, 5467.74 examples/s]


In [15]:

import os
os.environ["WANDB_DISABLED"] = "true"

everyweek 
2 research paper
1 GenAI - FineTuning -> ash Rajesh for weekly target



In [16]:
trainer.train()

/Users/sidharthrao/Documents/Documents - Sidharth’s MacBook Pro/GitHub/Project-Dash/2. GenAI and AgenticAI/ii. Class Sessions/n. Advanced FineTuning/.venv_AdvFT/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


TrainOutput(global_step=5, training_loss=1.8463024139404296, metrics={'train_runtime': 1.6928, 'train_samples_per_second': 5.907, 'train_steps_per_second': 2.954, 'total_flos': 2348828762112.0, 'train_loss': 1.8463024139404296})